In [1]:
import pandas as pd
from helper import DFA
from helper import setSolutionDFA

In [2]:
def lexerAritmetico(archivo):
    # Open file and parse content into list of expressions
    with open(archivo) as file:
        expressions = []
        line = file.readline()
        while line:
            expressions.append(line.strip())
            line = file.readline()

    # Visualize arithmetic expressions
    # print("\nArithmetic expressions\n--------------------------")
    # for exp in expressions: print(exp)
    # print("--------------------------")

    # Initialize DFA and its properties
    dfa = DFA()
    setSolutionDFA(dfa)

    # Run DFA with expressions
    accepted = dfa.execute_list(expressions, visualize=True, parse=True)
    # print(accepted)

    # Instead of returning nothing, as instructed, we return the DFA object
    # so that we can check the automata after the executions
    return dfa


In [3]:
archivo = "expresiones.txt"
dfa = lexerAritmetico(archivo)


Executing word: "b=7"
The word is accepted by the language
[('b', 'var'), ('=', '='), ('7', 'int')]


Executing word: "a = 32.4 (-8.6 - b)/       6.1E-8"
---------------------------------------
Syntax error at index 9: '('
Last state recorded: [await_op]
a = 32.4 (-8.6 - b)/       6.1E-8
         ^ Error
---------------------------------------
The word is not accepted by the language
[('a', 'var'), ('=', '='), ('32.4', 'float')]


Executing word: "d = a ^ b // Esto es un comentario"
The word is accepted by the language
[('d', 'var'), ('=', '='), ('a', 'var'), ('^', '^'), ('b', 'var'), ('// Esto es un comentario', 'comment')]



In [4]:
# Convert transition dictionary into a transition table (dataframe) of M states x N characters
transitions = pd.DataFrame(dfa.d)

# Replace NaN transitions with the default value of each state
for state, defdict in dfa.d.items():
    transitions[state].fillna(value=defdict.default_factory(), inplace=True)

# Sort row and column indices in alphabetic order for better visualization
transitions = transitions.sort_index().T.sort_index().T

# Show states and transitions
transitions

,(,),*,+,-,/,=,^,await_op,await_value,...,invalid,s,sign,single_dot,start_(,start_await_comment,success,var,var_),var_await_op
\n,invalid,success,invalid,invalid,invalid,invalid,invalid,invalid,success,invalid,...,invalid,success,invalid,invalid,success,invalid,success,success,success,success
,await_value,),await_value,await_value,await_value,await_value,await_value,await_value,),await_value,...,invalid,start_(,invalid,invalid,start_(,invalid,success,var_await_op,var_),var_)
(,(,invalid,(,(,(,(,(,(,invalid,(,...,invalid,start_(,invalid,invalid,start_(,invalid,success,invalid,invalid,invalid
),),),invalid,invalid,invalid,invalid,invalid,invalid,),invalid,...,invalid,invalid,invalid,invalid,invalid,invalid,success,var_),var_),var_)
*,invalid,*,invalid,invalid,invalid,invalid,invalid,invalid,*,invalid,...,invalid,invalid,invalid,invalid,invalid,invalid,success,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
v,var,invalid,var,var,var,var,var,var,invalid,var,...,invalid,var,invalid,invalid,var,invalid,success,var,invalid,invalid
w,var,invalid,var,var,var,var,var,var,invalid,var,...,invalid,var,invalid,invalid,var,invalid,success,var,invalid,invalid
x,var,invalid,var,var,var,var,var,var,invalid,var,...,invalid,var,invalid,invalid,var,invalid,success,var,invalid,invalid
y,var,invalid,var,var,var,var,var,var,invalid,var,...,invalid,var,invalid,invalid,var,invalid,success,var,invalid,invalid
